<a href="https://colab.research.google.com/github/Da-Heon/MachineLearning/blob/main/%EC%9E%A5%EB%B0%94%EA%B5%AC%EB%8B%88_%EC%95%8C%EA%B3%A0%EB%A6%AC%EC%A6%98.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 지지도, 신뢰도, 향상도

- 연관 규칙을 만들 때는 (지신향) 같은 지표로 후보 규칙을 선택
- 앞 예 데이터: "우유를 사면 달걀을 살 것이다(우유 > 달걍)"....
    - 가능한 모든 조합을 탐색(Brute Force)
    - 데이터가 늘어날수록 연산량이 기하급수적으로 증가
- 이러한 개선 폭발 문제를 해결하기 위한 알고리즘
- Apriori
    - "빈발하지 않은 항목을 포함한 상위 항목 집합은 빈발할 수 없다"
    - 자주 안팔리는 상품이 끼어 있으면, 그 상품이 들어간 어떤 묶음도 자주 팔릴 일이 없다
    - 후보 규칙을 대폭 줄여 효율적으로 연관 규칙을 찾을 수 있음

## Apriori

- 지지도가 낮은(자주 나타나지 않은) 조합을 한 번이라도 발견하면
- 그 조합을 포함하는 모든 상위 조합을 제외시켜 연산 효율서 증가
- "지지도"라는 기준으로 거래를 단계별로 스캔하면서 최소 지지도(Min Support) 이상을 만족하는 조합만 선택
- 여기서 살아남은 조합을 '빈발 항목 집합'이라 부름

In [1]:
import pandas as pd

In [3]:
df= pd.read_csv('/content/retail_data.csv')
basket_df = df.groupby('OrderID')['ProdName'].apply(list).reset_index()
basket_df.head()

,OrderID,ProdName
0,536365,"[WHITE HANGING HEART T-LIGHT HOLDER, WHITE MET..."
1,536366,"[HAND WARMER UNION JACK, HAND WARMER RED POLKA..."
2,536367,"[ASSORTED COLOUR BIRD ORNAMENT, POPPY'S PLAYHO..."
3,536368,"[JAM MAKING SET WITH JARS, RED COAT RACK PARIS..."
4,536369,[BATH BUILDING BLOCK WORD]


## 장바구니 원핫 인코딩

In [4]:
# 장바구니 형태 데이터를 원핫행렬로 바꿔주는 도구
# mlxtend: 장바구니 분석 라이브러리
from mlxtend.preprocessing import TransactionEncoder

In [8]:
data = [['우유', '빵'], ['우유', '달걀', '빵']]
te = TransactionEncoder()
te_ary = te.fit(data).transform(data)

In [9]:
df = pd.DataFrame(te_ary, columns=te.columns_)
df

,달걀,빵,우유
0,False,True,True
1,True,True,True


In [10]:
te = TransactionEncoder()
te_result = te.fit_transform(basket_df['ProdName'])
te_result

array([[False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       ...,
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False]])

In [11]:
te_df = pd.DataFrame(te_result, columns=te.columns_)
te_df

,4 PURPLE FLOCK DINNER CANDLES,SET 2 TEA TOWELS I LOVE LONDON,10 COLOUR SPACEBOY PEN,12 COLOURED PARTY BALLOONS,12 DAISY PEGS IN WOOD BOX,12 MESSAGE CARDS WITH ENVELOPES,12 PENCIL SMALL TUBE WOODLAND,12 PENCILS SMALL TUBE RED RETROSPOT,12 PENCILS SMALL TUBE SKULL,12 PENCILS TALL TUBE POSY,...,"WRAP, BILLBOARD FONTS DESIGN",YELLOW BREAKFAST CUP AND SAUCER,YELLOW COAT RACK PARIS FASHION,YELLOW GIANT GARDEN THERMOMETER,YELLOW SHARK HELICOPTER,YOU'RE CONFUSING ME METAL SIGN,YULETIDE IMAGES GIFT WRAP SET,ZINC FINISH 15CM PLANTER POTS,ZINC METAL HEART DECORATION,ZINC WILLIE WINKIE CANDLE STICK
0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,...,False,False,True,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
265,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
266,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,True,False,False,False
267,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
268,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [12]:
# 270건의 결제 건
# 1343개의 상품의 존재

### Apriori 알고리즘 학습

In [13]:
from mlxtend.frequent_patterns import apriori

In [14]:
df

,달걀,빵,우유
0,False,True,True
1,True,True,True


- min_support=0.05 : 최소시시도 설정
    - 전체 거래 중에서 최소 5% 이상 등장하는 상품의 묶음만
    - 자주 산다(빈발 항목 집합)고 인정하겠다는 뜻

In [17]:
# 지지도
apriori(df,
        min_support=0.05,
        use_colnames=True)

,support,itemsets
0,0.5,(달걀)
1,1.0,(빵)
2,1.0,(우유)
3,0.5,"(빵, 달걀)"
4,0.5,"(달걀, 우유)"
5,1.0,"(빵, 우유)"
6,0.5,"(빵, 달걀, 우유)"


In [21]:
# 빈발 항목 집합
freq_items = apriori(
    te_df,
    min_support=0.06,
    use_colnames=True
)

### 연관 규칙 추출 및 평가

In [22]:
from mlxtend.frequent_patterns import association_rules

In [23]:
test_freq_df = apriori(df, min_support=0.05, use_colnames=True)
test_freq_df

,support,itemsets
0,0.5,(달걀)
1,1.0,(빵)
2,1.0,(우유)
3,0.5,"(빵, 달걀)"
4,0.5,"(달걀, 우유)"
5,1.0,"(빵, 우유)"
6,0.5,"(빵, 달걀, 우유)"


In [25]:
use_cols = ['antecedents',   # IF 원인
            'consequents',   # THEN 결과
            'support',
            'confidence',
            'lift'
]

association_rules(test_freq_df,
                  metric='confidence',
                  min_threshold=0.8)[use_cols]

/usr/local/lib/python3.11/dist-packages/mlxtend/frequent_patterns/association_rules.py:186: RuntimeWarning: invalid value encountered in divide
  cert_metric = np.where(certainty_denom == 0, 0, certainty_num / certainty_denom)


,antecedents,consequents,support,confidence,lift
0,(달걀),(빵),0.5,1.0,1.0
1,(달걀),(우유),0.5,1.0,1.0
2,(빵),(우유),1.0,1.0,1.0
3,(우유),(빵),1.0,1.0,1.0
4,"(빵, 달걀)",(우유),0.5,1.0,1.0
5,"(달걀, 우유)",(빵),0.5,1.0,1.0
6,(달걀),"(빵, 우유)",0.5,1.0,1.0


- 달걀을 산다고 해서 빵을 사지는 않는다
- 빵과 달걀을 사는 조합이 있는데 우유를 사더라! 2건 중 한건은 이렇던데 그 비율이 100%

- min_support=0.06
- min_threshold=0.8
- 두 조건을 모두 통과한 규칙을 반환한 데이터프레임
- apriori() 가 먼저 빈발 아이템 집합을 구하고
- 각 아이템 집합에서 가능한 모든 'Ant => Cons'
- 그 중 신뢰도가 0.8 이상인 것만 남긴 결과

In [27]:
rules = association_rules(
    freq_items,
    metric='confidence',
    min_threshold=0.8
)
rules[use_cols].head()

,antecedents,consequents,support,confidence,lift
0,(GLASS STAR FROSTED T-LIGHT HOLDER),(KNITTED UNION FLAG HOT WATER BOTTLE),0.062963,0.944444,11.086957
1,(GLASS STAR FROSTED T-LIGHT HOLDER),(RED WOOLLY HOTTIE WHITE HEART.),0.062963,0.944444,9.107143
2,(GLASS STAR FROSTED T-LIGHT HOLDER),(WHITE HANGING HEART T-LIGHT HOLDER),0.066667,1.000000,7.297297
3,(GLASS STAR FROSTED T-LIGHT HOLDER),(WHITE METAL LANTERN),0.062963,0.944444,14.166667
4,(WHITE METAL LANTERN),(GLASS STAR FROSTED T-LIGHT HOLDER),0.062963,0.944444,14.166667


In [29]:
# antecedents(if: 조건)이 세트가 등장하면
# consequents(then: 결과) 애도 같이 자주 등장한다

# 전체 거래 중 약 6%가
#antecedents를 샀을 때, consequents도 사더라! 그 확률이 100%(conf)
# 이 일이 우연은 아니다(lift > 1, 15)

rules = rules[use_cols]
rules.sort_values(by='lift', ascending=False).head()

,antecedents,consequents,support,confidence,lift
156,"(WHITE METAL LANTERN, RED WOOLLY HOTTIE WHITE ...","(WHITE HANGING HEART T-LIGHT HOLDER, GLASS STA...",0.062963,1.0,15.882353
152,"(WHITE METAL LANTERN, KNITTED UNION FLAG HOT W...","(WHITE HANGING HEART T-LIGHT HOLDER, GLASS STA...",0.062963,1.0,15.882353
151,"(GLASS STAR FROSTED T-LIGHT HOLDER, KNITTED UN...","(WHITE HANGING HEART T-LIGHT HOLDER, WHITE MET...",0.062963,1.0,15.882353
138,"(WHITE HANGING HEART T-LIGHT HOLDER, WHITE MET...","(GLASS STAR FROSTED T-LIGHT HOLDER, RED WOOLLY...",0.062963,1.0,15.882353
137,"(WHITE HANGING HEART T-LIGHT HOLDER, GLASS STA...","(WHITE METAL LANTERN, RED WOOLLY HOTTIE WHITE ...",0.062963,1.0,15.882353
